# Flujo de Autenticación con JWT

## Construir un API REST con Django REST Framework

Crea un ambiente virtual:
```
python3 -m venv env
```
Activa el ambiente virtual:
```
# Activación en Unix
source env/bin/activate

# Activación en Windows
env\Scripts\activate
```
Instala Django, DRF y PyJWT:
```
pip install django
pip install djangorestframework
pip install pyjwt
```
Crea un nuevo proyecto en Django:
```
django-admin startproject jwt_auth
```
Crea una nueva aplicación en Django:
```
python manage.py startapp api
```
Agrega la aplicación de `rest_framework` y la que acabamos de crear en el archivo de `settings.py`:

In [ ]:
# Modificamos jwt_auth/settings.py

# Especificar qué aplicaciones están habilitadas y disponibles para ser usadas por el proyecto.
INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'rest_framework',
    'api'
]

## Registro de usuarios

### Define un serializador para el modelo User



In [ ]:
# Creamos api/serializers.py

# Importa el modelo User de Django, que es el modelo de usuario predeterminado utilizado para la autenticación y la gestión de usuarios en proyectos de Django.
from django.contrib.auth.models import User
# Importa el módulo serializers de Django REST Framework.
from rest_framework import serializers


# Define una nueva clase UserSerializer.
class UserSerializer(serializers.ModelSerializer):
    # Esta subclase Meta se utiliza para especificar opciones de metadatos para el serializador UserSerializer.
    class Meta:
        # Indica que el serializador está asociado con el modelo User.
        model = User
        # Especifica una lista de nombres de campos del modelo User que se incluirán en el serializado/deserializado.
        fields = ["username", "email", "password"]
        # Esta línea establece argumentos adicionales para los campos especificados en fields.
        extra_kwargs = {"password": {"write_only": True}}

### Define las vistas diseñadas para manejar operaciones CRUD (Crear, Leer, Actualizar, Borrar) sobre usuarios.

In [ ]:
# Modificamos api/views.py

# Importa el modelo User, el modelo de usuario predeterminado de Django.
from django.contrib.auth.models import User
# Importa el módulo status que contiene códigos de estado HTTP para las respuestas.
from rest_framework import status
# Importa la clase Response que se utiliza para retornar respuestas desde las vistas de DRF.
from rest_framework.response import Response
# Importa APIView, la clase base para todas las vistas en DRF.
from rest_framework.views import APIView
# Importa el serializador UserSerializer.
from .serializers import UserSerializer


# Define una vista UserViews
class UserViews(APIView):
    # Establece UserSerializer como el serializador por defecto para esta vista.
    serializer_class = UserSerializer

    # Define cómo manejar GET, para listar todos los usuarios.
    def get(self, request):
        # Obtiene todos los usuarios de la base de datos.
        users = User.objects.all()
        # Serializa la lista de usuarios utilizando UserSerializer
        serializer = UserSerializer(users, many=True)
        # Retorna una respuesta indicando el éxito de la operación.
        return Response(
            {"status": "OK", "data": serializer.data},
            status=status.HTTP_200_OK,
        )

    # Define cómo manejar POST, para crear un nuevo usuario.
    def post(self, request):
        # Crea una instancia de UserSerializer con los datos enviados en la solicitud.
        serializer = UserSerializer(data=request.data)
        # Verifica si los datos son válidos.
        if serializer.is_valid():
            # Guarda el usuario en la base de datos.
            serializer.save()
            # Retorna una respuesta indicando el éxito de la operación.
            return Response(
                {"status": "CREATED", "data": serializer.data},
                status=status.HTTP_201_CREATED,
            )
        # Retorna una respuesta indicando el fallo de la operación.
        return Response(
            {"status": "BAD_REQUEST", "data": serializer.errors},
            status=status.HTTP_400_BAD_REQUEST,
        )


# Define una vista UserDetailViews
class UserDetailViews(APIView):
    # Establece UserSerializer como el serializador por defecto para esta vista.
    serializer_class = UserSerializer

    # Define cómo manejar GET, para obtener detalles de un usuario específico.
    def get(self, request, username):
        # Recupera una instancia del modelo User
        user = User.objects.get_by_natural_key(username)
        # Crea una instancia de UserSerializer, pasando el usuario obtenido como el objeto a serializar.
        serializer = UserSerializer(user)
        # # Retorna una respuesta indicando el éxito de la operación.
        return Response(
            {"status": "OK", "data": serializer.data},
            status=status.HTTP_200_OK,
        )

    # Define cómo manejar PATCH, para actualizar parcialmente recursos.
    def patch(self, request, username):
        # Recupera una instancia del modelo User
        user = User.objects.get_by_natural_key(username)
        # Crea una instancia de UserSerializer, esta vez pasando tanto el objeto user como los datos provenientes de la solicitud.
        serializer = UserSerializer(user, data=request.data, partial=True)
        # Verifica si los datos son válidos.
        if serializer.is_valid():
            # Guarda el usuario en la base de datos.
            serializer.save()
            # Retorna una respuesta indicando el éxito de la operación.
            return Response(
                {"status": "OK", "data": serializer.data},
                status=status.HTTP_200_OK,
            )
        # Retorna una respuesta indicando el fallo de la operación.
        return Response(
            {"status": "BAD_REQUEST", "data": serializer.errors},
            status=status.HTTP_400_BAD_REQUEST,
        )

    # Define cómo manejar DELETE, para eliminar un usuario específico.
    def delete(self, request, username):
        # Recupera una instancia del modelo User
        user = User.objects.get_by_natural_key(username)
        # Elimina el usuario encontrado de la base de datos.
        user.delete()
        # Retorna una respuesta indicando el éxito de la operación.
        return Response(
            {"status": "OK", "data": "User deleted."},
            status=status.HTTP_200_OK,
        )

### Asigna URLs específicas a vistas definidas en la aplicación.

In [ ]:
# Creamos api/urls.py

# Importa la función path del módulo django.urls, que se utiliza para definir patrones de URL individuales.
from django.urls import path
# Importa las vistas UserDetailViews y UserViews.
from .views import UserDetailViews, UserViews

# Cada elemento de esta lista es una llamada a la función path, que define una asociación entre un patrón de URL y una vista.
urlpatterns = [
    path("users/", UserViews.as_view()),
    path("users/<str:username>", UserDetailViews.as_view()),
]

### Configura las rutas URL en el proyecto Django

In [ ]:
# Modificamos jwt_auth/urls.py

# Importa el módulo de administración de Django.
from django.contrib import admin
# Importa las funciones path e include del módulo django.urls.
# path se utiliza para definir patrones de URL individuales.
# include permite referenciar definiciones de URL de otras aplicaciones Django.
from django.urls import path, include

# Cada elemento de esta lista es una llamada a la función path, que define una asociación entre un patrón de URL y una vista.
urlpatterns = [
    path("admin/", admin.site.urls),
    path("api/", include('api.urls')),
]


## Inicio de sesión y Generación de JWT

Crea una nueva aplicación en Django:
```
python manage.py startapp accounts
```
Agrega la aplicación de `rest_framework` y la que acabamos de crear en el archivo de `settings.py`:

In [ ]:
# Modificamos jwt_auth/settings.py

# Especificar qué aplicaciones están habilitadas y disponibles para ser usadas por el proyecto.
INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'rest_framework',
    'api',
    'accounts',
]

### Define un serializador para el Log in

In [ ]:
# Creamos accounts/serializers.py

# Importa el módulo serializers de Django REST Framework.
from rest_framework import serializers


# Define una nueva clase LoginSerializer.
class LoginSerializer(serializers.Serializer):
    # Se define un campo username
    username = serializers.CharField()
    # Se define un campo password
    password = serializers.CharField()

### Define una función que genera un token JWT para un usuario específico.

In [ ]:
# Creamos accounts/utils.py

# Importa las clases datetime y timedelta del módulo datetime. datetime se utiliza para obtener el tiempo actual en formato UTC, y timedelta para definir un periodo de tiempo.
from datetime import datetime, timedelta

# Importa el módulo jwt.
import jwt
# Importa el módulo settings de Django.
from django.conf import settings
# Importa el modelo User
from django.contrib.auth.models import User
from django.contrib.auth import authenticate


# Generar y retornar un token JWT para el usuario proporcionado.
def generate_jwt_token(user: User):
    # Contiene las reclamaciones (claims) que se incluirán en el token JWT.
    payload = {
        "user_pk": user.pk,
        "username": user.username,
        "exp": datetime.utcnow() + timedelta(hours=1),
    }
    # Retorna el resultado de codificar el payload en un token JWT
    return jwt.encode(payload, settings.SECRET_KEY, algorithm="HS256")


### Define una vista para manejar solicitudes de inicio de sesión y generar tokens JWT para usuarios autenticados.

In [ ]:
# Modificamos accounts/views.py

# Importa el módulo status que contiene códigos de estado HTTP para las respuestas.
from rest_framework import status
# Importa la clase Response que se utiliza para retornar respuestas desde las vistas de DRF.
from rest_framework.response import Response
# Importa APIView, la clase base para todas las vistas en DRF.
from rest_framework.views import APIView

# Importa LoginSerializer
from .serializers import LoginSerializer
# Importa la función generate_jwt_token
from .utils import generate_jwt_token


# Define una vista LoginViews
class LoginView(APIView):
    # Establece UserSerializer como el serializador por defecto para esta vista.
    serializer_class = LoginSerializer

    # Define cómo manejar POST, para enviar datos de inicio de sesión.
    def post(self, request):
        # Crea una instancia de LoginSerializer con los datos enviados en la solicitud.
        serializer = LoginSerializer(data=request.data)
        # Verifica si los datos son válidos.
        if serializer.is_valid():
            # Obtiene el nombre de usuario de los datos validados.
            username = serializer.validated_data["username"]
            # Obtiene la contraseña de los datos validados.
            password = serializer.validated_data["password"]
            # Intenta autenticar al usuario utilizando la función authenticate.
            user = authenticate(request, username=username, password=password)
            # Verifica si la autenticación fue exitosa.
            if user is not None:
                # Genera un token JWT para el usuario
                token = generate_jwt_token(user)
                # Retorna una respuesta indicando el éxito de la operación.
                return Response(
                    {"status": "OK", "access_token": token},
                    status=status.HTTP_200_OK,
                )
            # Retorna una respuesta indicando el fallo de la operación.
            return Response(
                {
                    "status": "UNAUTHORIZED",
                    "data": "The username or password you are trying to log in"
                    " with are incorrect.",
                },
                status=status.HTTP_401_UNAUTHORIZED,
            )
        # Retorna una respuesta indicando el fallo de la operación.
        return Response(
            {"status": "BAD_REQUEST", "data": serializer.errors},
            status=status.HTTP_400_BAD_REQUEST,
        )

### Asigna URLs específicas a vistas nuevas definidas en la aplicación.

In [ ]:
# Creamos accounts/urls.py

# Importa las vistas LoginView.
from .views import LoginView

# Cada elemento de esta lista es una llamada a la función path, que define una asociación entre un patrón de URL y una vista.
urlpatterns = [
    path("login/", LoginView.as_view()),
]

### Configura las rutas URL en el proyecto Django

In [ ]:
# Modificamos jwt_auth/urls.py

# Importa el módulo de administración de Django.
from django.contrib import admin
# Importa las funciones path e include del módulo django.urls.
# path se utiliza para definir patrones de URL individuales.
# include permite referenciar definiciones de URL de otras aplicaciones Django.
from django.urls import path, include

# Cada elemento de esta lista es una llamada a la función path, que define una asociación entre un patrón de URL y una vista.
urlpatterns = [
    path("admin/", admin.site.urls),
    path("api/", include('api.urls')),
    path("", include("accounts.urls")),
]


## Validación y Verificación de JWT

### Define un middleware personalizado en Django para manejar la autenticación JWT en las solicitudes HTTP entrantes a la aplicación.

In [ ]:
# Creamos accounts/middlewares.py

# Define un middleware personalizado en Django para manejar la autenticación JWT (JSON Web Token) en las solicitudes HTTP entrantes a la aplicación.
import jwt
# Importa la configuración del proyecto Django.
from django.conf import settings
# Importa JsonResponse, que es una forma conveniente de devolver respuestas JSON desde una vista Django.
from django.http import JsonResponse
# Importa el módulo status de Django REST Framework.
from rest_framework import status
# Importa la clase Request de Django REST Framework
from rest_framework.request import Request


# Define la función jwt_auth_middleware, que acepta un argumento get_response.
# get_response es una función de callback que Django llama con el objeto request para obtener la respuesta de la vista o del siguiente middleware.
def jwt_auth_middleware(get_response):

    def middleware(request: Request):
        # Comprueba si la cabecera HTTP Authorization está presente en la solicitud.
        if "HTTP_AUTHORIZATION" in request.META:
            # Extrae el valor de la cabecera Authorization.
            auth_header = request.META["HTTP_AUTHORIZATION"]
            try:
                # Divide el valor de la cabecera para separar el tipo de autenticación (generalmente "Bearer") del token propiamente dicho.
                _, token = auth_header.split()
                # Intenta decodificar el token JWT usando la clave secreta del proyecto y el algoritmo 'HS256'.
                request.decoded_jwt = jwt.decode(
                    token, settings.SECRET_KEY, algorithms=["HS256"]
                )
                # Si el token es válido, permite que la solicitud continúe hacia la vista o el siguiente middleware.
                return get_response(request)
            # Captura y maneja el error si el token JWT ha expirado.
            except jwt.ExpiredSignatureError:
                return JsonResponse(
                    {
                        "status": "OK",
                        "data": "The token has expired. Please log in again to"
                        " obtain a new token.",
                    },
                    status=status.HTTP_401_UNAUTHORIZED,
                )
            # Captura y maneja el error si el token JWT es inválido.
            except jwt.InvalidTokenError:
                return JsonResponse(
                    {
                        "status": "OK",
                        "data": "The provided token is invalid. Please check "
                        "the token and try again.",
                    },
                    status=status.HTTP_401_UNAUTHORIZED,
                )
        # Si no se proporciona un token, retorna una respuesta JSON con un mensaje indicando que se requiere un token de autenticación.
        else:
            return JsonResponse(
                {
                    "status": "OK",
                    "data": "An authentication token is required to access "
                    "this resource.",
                },
                status=status.HTTP_401_UNAUTHORIZED,
            )
    # Retorna la función middleware configurada.
    return middleware

### Agrega el middleware que acabamos de crear en el archivo de `settings.py`:

In [ ]:
# Modificamos jwt_auth/settings.py

# Especificar qué middlewares están habilitados y disponibles para ser usadas por el proyecto.

MIDDLEWARE = [
    "django.middleware.security.SecurityMiddleware",
    "django.contrib.sessions.middleware.SessionMiddleware",
    "django.middleware.common.CommonMiddleware",
    "django.middleware.csrf.CsrfViewMiddleware",
    "django.contrib.auth.middleware.AuthenticationMiddleware",
    "django.contrib.messages.middleware.MessageMiddleware",
    "django.middleware.clickjacking.XFrameOptionsMiddleware",
    'accounts.middleware.jwt_auth_middleware',
]

### Modificamos middleware para agregar las rutas de URL para las cuales no debe aplicar la lógica de autenticación JWT.

In [ ]:
# Modificamos accounts/middlewares.py

# Define la función jwt_auth_middleware, que acepta un argumento get_response.
# get_response es una función de callback que Django llama con el objeto request para obtener la respuesta de la vista o del siguiente middleware.
def jwt_auth_middleware(get_response):
    # Define una lista de endpoints llamada excluded_endpoints que contiene las rutas de URL para las cuales este middleware no debe aplicar la lógica de autenticación JWT.
    excluded_endpoints = ["/login/"]

    def middleware(request: Request):
        # Este bloque condicional verifica si la ruta de la solicitud actual está en la lista de excluded_endpoints.
        # Si es así, el middleware permite que la solicitud continúe sin procesamiento adicional.
        if request.path in excluded_endpoints:
            return get_response(request)

        # Comprueba si la cabecera HTTP Authorization está presente en la solicitud.
        if "HTTP_AUTHORIZATION" in request.META:
            # Extrae el valor de la cabecera Authorization.
            auth_header = request.META["HTTP_AUTHORIZATION"]

## Manejo de tokens de actualización y refresco

### Modificamos la función que genera un token JWT para crear y retornar dos tokens JWT: un "access token" y un "refresh token" para un usuario específico

In [ ]:
# Modificamos accounts/utils.py

# Importa las clases datetime y timedelta del módulo datetime. datetime se utiliza para obtener el tiempo actual en formato UTC, y timedelta para definir un periodo de tiempo.
from datetime import datetime, timedelta

# Importa el módulo jwt.
import jwt
# Importa el módulo settings de Django.
from django.conf import settings
# Importa el modelo User
from django.contrib.auth.models import User


# Generar y retornar dos token JWT para el usuario proporcionado.
def generate_jwt_tokens(user: User):
    # Contiene las claims que se incluirán en el JWT para el "access token".
    access_payload = {
        "user_pk": user.pk,
        "username": user.username,
        "exp": datetime.utcnow() + timedelta(hours=1),
    }
    # Utiliza la función encode del módulo jwt para codificar el access_payload en un JWT.
    access_token = jwt.encode(
        access_payload, settings.SECRET_KEY, algorithm="HS256"
    )

    # Contiene las claims que se incluirán en el JWT para el "refresh token".
    refresh_payload = {
        "user_pk": user.pk,
        "username": user.username,
        "exp": datetime.utcnow() + timedelta(days=1),
    }
    # Utiliza la función encode del módulo jwt para codificar el refresh_payload en un JWT.
    refresh_token = jwt.encode(
        refresh_payload, f"refresh.{settings.SECRET_KEY}", algorithm="HS256"
    )
    # La función retorna una tupla que contiene el "access token" y el "refresh token".
    return access_token, refresh_token

### Define un serializador para el refresh token

In [ ]:
# Modificamos accounts/serializers.py


# Define una nueva clase RefreshTokenSerializer.
class RefreshTokenSerializer(serializers.Serializer):
    # Se define un campo refresh_token
    refresh_token = serializers.CharField()

### Modifica la vista de inicio de sesión para regresar los dos tokens y Define una vista para refrescar el token.

In [ ]:
# Modificamos accounts/views.py

# Importa la biblioteca PyJWT, utilizada para decodificar y verificar tokens JWT.
import jwt
# Importa las configuraciones de Django para acceder a variables como SECRET_KEY.
from django.conf import settings
# Importa la función de autenticación de Django, aunque no se utiliza directamente en este fragmento.
from django.contrib.auth import authenticate
# Importa el modelo de usuario predeterminado de Django.
from django.contrib.auth.models import User
# Importa el módulo status que contiene códigos de estado HTTP para las respuestas.
from rest_framework import status
# Importa la clase Request de DRF que encapsula la solicitud HTTP.
from rest_framework.request import Request
# Importa la clase Response que se utiliza para retornar respuestas desde las vistas de DRF.
from rest_framework.response import Response
# Importa APIView, la clase base para todas las vistas en DRF.
from rest_framework.views import APIView

# Importa LoginSerializer y RefreshTokenSerializer
from .serializers import LoginSerializer, RefreshTokenSerializer
# Importa la función generate_jwt_token
from .utils import generate_jwt_token


# Define una vista LoginViews
class LoginView(APIView):
    # Establece LoginSerializer como el serializador por defecto para esta vista.
    serializer_class = LoginSerializer

    # Define cómo manejar POST, para enviar datos de inicio de sesión.
    def post(self, request):
        # Crea una instancia de LoginSerializer con los datos enviados en la solicitud.
        serializer = LoginSerializer(data=request.data)
        # Verifica si los datos son válidos.
        if serializer.is_valid():
            # Obtiene el nombre de usuario de los datos validados.
            username = serializer.validated_data["username"]
            # Obtiene la contraseña de los datos validados.
            password = serializer.validated_data["password"]
            # Intenta autenticar al usuario utilizando la función authenticate.
            user = authenticate(request, username=username, password=password)
            # Verifica si la autenticación fue exitosa.
            if user is not None:
                # Genera los dos tokens JWT para el usuario
                access_token, refresh_token = generate_jwt_tokens(user)
                # Retorna una respuesta indicando el éxito de la operación.
                return Response(
                    {
                        "status": "OK",
                        "access_token": access_token,
                        "refresh_token": refresh_token,
                    },
                    status=status.HTTP_200_OK,
                )
            # Retorna una respuesta indicando el fallo de la operación.
            return Response(
                {
                    "status": "UNAUTHORIZED",
                    "data": "The username or password you are trying to log in"
                    " with are incorrect.",
                },
                status=status.HTTP_401_UNAUTHORIZED,
            )
        # Retorna una respuesta indicando el fallo de la operación.
        return Response(
            {"status": "BAD_REQUEST", "data": serializer.errors},
            status=status.HTTP_400_BAD_REQUEST,
        )


# # Define una vista LoginViews
class RefreshTokenView(APIView):
    # Establece RefreshTokenSerializer como el serializador por defecto para esta vista.
    serializer_class = RefreshTokenSerializer

    # Define cómo manejar POST, para enviar datos de refresco de token.
    def post(self, request: Request):
        # Crea una instancia del serializador con los datos proporcionados en la solicitud.
        serializer = RefreshTokenSerializer(data=request.data)
        # Verifica si los datos son válidos.
        if serializer.is_valid():
            # Obtiene el refresh token validado desde los datos del serializador.
            refresh_token = serializer.validated_data["refresh_token"]
            try:
                # Intenta decodificar el refresh token utilizando la clave secreta y el algoritmo especificado.
                payload = jwt.decode(
                    refresh_token,
                    f"refresh.{settings.SECRET_KEY}",
                    algorithms=["HS256"],
                )
                # Extrae el nombre de usuario del payload del token.
                username = payload["username"]
                # Busca el usuario en la base de datos por su nombre natural (nombre de usuario).
                user = User.objects.get_by_natural_key(username)
                # Verifica si se encontró el usuario.
                if user is not None:
                    # Genera nuevos tokens de acceso y actualización.
                    access_token, refresh_token = generate_jwt_tokens(user)
                    # Devuelve una respuesta con los nuevos tokens.
                    return Response(
                        {
                            "status": "OK",
                            "access_token": access_token,
                            "refresh_token": refresh_token,
                        },
                        status=status.HTTP_200_OK,
                    )
            # Captura el error si el token ha expirado y devuelve un mensaje apropiado.
            except jwt.ExpiredSignatureError:
                return Response(
                    {
                        "status": "OK",
                        "data": "The token has expired. Please log in again to"
                        " obtain a new token.",
                    },
                    status=status.HTTP_401_UNAUTHORIZED,
                )
            # Captura el error si el token es inválido y devuelve un mensaje apropiado.
            except jwt.InvalidTokenError:
                return Response(
                    {
                        "status": "OK",
                        "data": "The provided token is invalid. Please check "
                        "the token and try again.",
                    },
                    status=status.HTTP_401_UNAUTHORIZED,
                )
        # Si los datos iniciales no son válidos o si ocurre cualquier otro error, devuelve un error de solicitud incorrecta con detalles.
        return Response(
            {"status": "BAD_REQUEST", "data": serializer.errors},
            status=status.HTTP_400_BAD_REQUEST,
        )

### Asigna URLs específicas a vistas nuevas definidas en la aplicación.

In [ ]:
# Creamos accounts/urls.py

# Importa las vistas LoginView.
from .views import LoginView

# Cada elemento de esta lista es una llamada a la función path, que define una asociación entre un patrón de URL y una vista.
urlpatterns = [
    path("login/", LoginView.as_view()),
    path("token/refresh/", RefreshTokenView.as_view()),
]

### Modificamos middleware para agregar las rutas de URL para las cuales no debe aplicar la lógica de autenticación JWT.

In [ ]:
# Modificamos accounts/middlewares.py

# Define la función jwt_auth_middleware, que acepta un argumento get_response.
# get_response es una función de callback que Django llama con el objeto request para obtener la respuesta de la vista o del siguiente middleware.
def jwt_auth_middleware(get_response):
    # Define una lista de endpoints llamada excluded_endpoints que contiene las rutas de URL para las cuales este middleware no debe aplicar la lógica de autenticación JWT.
    excluded_endpoints = ["/login/", "/token/refresh/"]

    def middleware(request: Request):
        # Este bloque condicional verifica si la ruta de la solicitud actual está en la lista de excluded_endpoints.
        # Si es así, el middleware permite que la solicitud continúe sin procesamiento adicional.
        if request.path in excluded_endpoints:
            return get_response(request)

        # Comprueba si la cabecera HTTP Authorization está presente en la solicitud.
        if "HTTP_AUTHORIZATION" in request.META:
            # Extrae el valor de la cabecera Authorization.
            auth_header = request.META["HTTP_AUTHORIZATION"]